In [1]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [2]:
type(documents)

list

In [3]:
len(documents)

11314

In [4]:
documents[45]

"*Reminder*   Plan now for the Andrew Conference.\n*Date* The dates are as noted below.  (We have not changed them.)\n*Submission extension*   We are still accepting papers.\n\n*Tutorial topic*  \n\t_Converting Andrew source code to C++_\n\nThis tutorial will discuss the steps necessary to convert a site from C\n(extended with classC) to C++.  Conversion of the source code requires\nonly a couple of steps:\n\trun the converter\n\tfill in missing type information\nDescribing this will not take long.  The remainder of the day will be\nspent learning how to write objects in C++ and practicing.\n\n------------------------------\n\n1993 Andrew Technical Conference and Consortium Annual Meeting\nJune 24-25, 1993\nCarnegie Mellon University\nPittsburgh, PA\n\nThe conference will be held on the last Thursday and Friday in June.  A\ntutorial will be on Thursday the 24th and the conference proper on the\n25th with the annual meeting at the dinner on the evening between the\ntwo days.  All confer

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
#keep only the 1000 most frequent words of the corpus that appear in more than 2 different docs and 
#in less than 95% of the docs
tf_vectorizer = CountVectorizer(stop_words="english", max_df=0.5, min_df=2, max_features=1000).fit(documents)
tf = tf_vectorizer.transform(documents)

In [30]:
tf_vectorizer.vocabulary_

{'sure': 872,
 'story': 860,
 'did': 303,
 'statement': 854,
 'media': 589,
 'pro': 714,
 'israeli': 497,
 'world': 985,
 'having': 444,
 'letter': 535,
 'try': 914,
 'think': 897,
 'reason': 745,
 'report': 760,
 'clearly': 229,
 'reports': 762,
 'received': 748,
 'government': 424,
 'makes': 573,
 'away': 146,
 'look': 554,
 'jews': 502,
 'got': 422,
 'power': 703,
 'expect': 359,
 'people': 670,
 'read': 741,
 'faq': 369,
 'actually': 88,
 'accept': 83,
 'hard': 441,
 'atheism': 138,
 'need': 620,
 'little': 546,
 'faith': 366,
 'runs': 781,
 'jim': 503,
 'sorry': 837,
 'oh': 644,
 'just': 508,
 'end': 341,
 'maybe': 584,
 'start': 851,
 'new': 625,
 'newsgroup': 627,
 'alt': 103,
 'won': 978,
 'big': 162,
 'don': 321,
 'points': 691,
 'like': 540,
 'know': 516,
 'ask': 134,
 'question': 734,
 'sort': 838,
 'want': 954,
 'continue': 258,
 'israel': 496,
 'stop': 859,
 'asking': 136,
 'questions': 735,
 'work': 981,
 'bad': 149,
 'attack': 139,
 'group': 429,
 'center': 203,
 'policy

In [28]:
type(tf)

scipy.sparse.csr.csr_matrix

In [27]:
tf.shape

(11314, 1000)

### LDA

In [29]:
from sklearn.decomposition import LatentDirichletAllocation

# Run LDA
lda = LatentDirichletAllocation(n_components=20, max_iter=5, learning_method='online', 
                                learning_offset=50.,random_state=0).fit(tf)

In [34]:
lda.components_

array([[5.00438141e-02, 8.78732335e+01, 5.00256800e-02, ...,
        5.66294140e+01, 5.00615966e-02, 3.45541732e+00],
       [5.03280790e-02, 2.75136704e-01, 5.12194373e-02, ...,
        7.58226968e+01, 5.43063786e-02, 6.40093199e-02],
       [5.00676581e-02, 5.00277733e-02, 5.00496354e-02, ...,
        1.60606909e+02, 5.00642090e-02, 5.00376473e-02],
       ...,
       [1.06478402e+03, 2.63324015e+02, 2.23045168e+02, ...,
        5.00564571e-02, 4.99493555e+01, 2.50369810e+01],
       [5.00451225e-02, 1.31202890e+02, 5.00235394e-02, ...,
        1.86296233e+00, 2.42802490e+01, 5.37686203e+01],
       [5.13812136e-02, 6.28433834e+01, 5.00675017e-02, ...,
        8.08756126e+00, 1.31918551e+02, 1.47488372e+02]])

In [35]:
def display_topics(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

display_topics(lda, tf_vectorizer.get_feature_names(), 10)

Topic 0:
people gun state control right guns crime states law police
Topic 1:
time question book years did like don space answer just
Topic 2:
mr line rules science stephanopoulos title current define int yes
Topic 3:
key chip keys clipper encryption number des algorithm use bit
Topic 4:
edu com cs vs w7 cx mail uk 17 send
Topic 5:
use does window problem way used point different case value
Topic 6:
windows thanks know help db does dos problem like using
Topic 7:
bike water effect road design media dod paper like turn
Topic 8:
don just like think know people good ve going say
Topic 9:
car new price good power used air sale offer ground
Topic 10:
file available program edu ftp information files use image version
Topic 11:
ax max b8f g9v a86 145 pl 1d9 0t 34u
Topic 12:
government law privacy security legal encryption court fbi technology information
Topic 13:
card bit memory output video color data mode monitor 16
Topic 14:
drive scsi disk mac hard apple drives controller software port
T

### NMF

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)

# Run NMF
nmf = NMF(n_components=20, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

display_topics(nmf, tfidf_vectorizer.get_feature_names(), 10)

Topic 0:
people time right did good said say make way government
Topic 1:
window problem using server application screen display motif manager running
Topic 2:
god jesus bible christ faith believe christian christians sin church
Topic 3:
game team year games season players play hockey win league
Topic 4:
new 00 sale 10 price offer shipping condition 20 15
Topic 5:
thanks mail advance hi looking info help information address appreciated
Topic 6:
windows file files dos program version ftp ms directory running
Topic 7:
edu soon cs university ftp internet article email pub david
Topic 8:
key chip clipper encryption keys escrow government public algorithm nsa
Topic 9:
drive scsi drives hard disk ide floppy controller cd mac
Topic 10:
just ll thought tell oh little fine work wanted mean
Topic 11:
does know anybody mean work say doesn help exist program
Topic 12:
card video monitor cards drivers bus vga driver color memory
Topic 13:
like sounds looks look bike sound lot things really thing
To